## Parse comments

In [4]:
import json, os, shutil, re, time, csv
from lxml import etree
from tqdm import tqdm

html_path = r"J:/EECS_6414/Data/19-02-24/kernel_split/htmls/0/aahamdan_titanic-first-notebook.html"

with open(html_path, "r", encoding="utf-8") as f:
    html = f.read()
page_source = etree.HTML(html)
##################
# sometimes, the html parsing can get none.
# just skip this kind of situation
##################
# if page_source == None:
#     continue
###########
# this part is for dealing with the problem
# of: too many request.
# we have to abondon those "too many requests" pages temprarily.
###########
# if page_source.xpath("/html/body/pre/text()") != []:
#     print("bad file: {}".format("{}/{}".format(num, file)))
#     continue
############
_ = page_source.xpath("//div[@class='site-layout__main-content']/script[1]/text()")[0]
p = re.compile("Kaggle\.State\.push\(\{[\s\S]*\}\)\;")

s = p.findall(_)[0].replace("Kaggle.State.push(", "")[0:-2]
info = json.loads(s)

In [5]:
info

{'kernel': {'id': 2051724,
  'title': 'Titanic first Notebook',
  'forkParent': None,
  'currentRunId': 10737600,
  'mostRecentRunId': 10830368,
  'url': '/aahamdan/titanic-first-notebook',
  'tags': [],
  'commentCount': 0,
  'upvoteCount': 2,
  'viewCount': 186,
  'forkCount': 2,
  'bestPublicScore': 0.73684,
  'author': {'id': 1339128,
   'displayName': 'Ahmad Hamdan',
   'email': None,
   'editedEmail': None,
   'editedEmailCode': None,
   'userName': 'aahamdan',
   'thumbnailUrl': 'https://storage.googleapis.com/kaggle-avatars/thumbnails/default-thumb.png',
   'profileUrl': '/aahamdan',
   'registerDate': '0001-01-01T00:00:00Z',
   'lastVisitDate': '0001-01-01T00:00:00Z',
   'statusId': 0,
   'performanceTier': 0,
   'userRoles': None,
   'userLogins': None,
   'groupIds': None,
   'duplicateUsers': None,
   'hasPhoneVerifications': False,
   'failedNerdchas': 0,
   'hasPendingNerdcha': False,
   'deleteRequests': None,
   'userAttributes': None,
   'isAdmin': False,
   'isTvc': F

# Code toys

In [1]:
!pip install pygount

# analyse the data

## calculating the target encoding for multiple tags

In [1]:
import pandas as pd
import os
# cvo: code_voted_original 
# cvp: code_voted_pyscript
# km: kernel_measures
merged_table = r"J:\EECS6414\process\Pipeline\kernel_datssource_code.csv"
mt = pd.read_csv(merged_table, encoding="utf-8")

In [2]:
mt["km_votes"].describe()

count    28205.000000
mean        11.483425
std         60.795352
min          0.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       3713.000000
Name: km_votes, dtype: float64

In [3]:
mt["high_voted"] = None
mt["high_voted"][mt["km_votes"] >= 2] = True
mt["high_voted"][mt["km_votes"] < 2] = False
mt["high_voted"].describe()

C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\xmk233\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count     28205
unique        2
top        True
freq      18604
Name: high_voted, dtype: object

In [4]:
kernel_tags = {}
tags_kernel = {}
tags_high_and_low_vote_kernel_number = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["km_tags"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        continue
    how_many_kernels_have_tags += 1
    tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
        #####
        if tag in tags_high_and_low_vote_kernel_number:
            if row["high_voted"]:
                tags_high_and_low_vote_kernel_number[tag][0] += 1
            else:
                tags_high_and_low_vote_kernel_number[tag][1] += 1
        else:
            tags_high_and_low_vote_kernel_number[tag] = [1, 0] if row["high_voted"] else [0, 1] 
    kernel_tags[kernel_id] = tags_of_this_kernel
            
print(
    "how many kernels have tags: ", how_many_kernels_have_tags, "\n",  
    "how_many_kernels_dont_have_tags: ", how_many_kernels_dont_have_tags, "\n"
    #"how many tags are in total: ", len(list(tags_num.keys())), "\n"
)
def target_encoding_tags(string):
    import numpy as np
    if isinstance(string, float):
        return 0
    t = tags_high_and_low_vote_kernel_number
    tags = string.split(";")
    if len(tags) == 0:
        return 0
    sum_of_encoding = 0
    for tag in string.split(";"):
        sum_of_encoding += float(t[tag][0]) / sum(t[tag])
    return sum_of_encoding / len(tags)
mt["km_target_encoding"] = 0
mt["km_target_encoding"] = mt["km_tags"].map(target_encoding_tags)

mt.to_csv("kernel_datssource_code_0.csv", index=None)

how many kernels have tags:  6075 
 how_many_kernels_dont_have_tags:  22130 



## calculate target encoding for languages

## getting all the tags

In [14]:
kernel_tags = {}
tags_kernel = {}

how_many_kernels_dont_have_tags = 0
how_many_kernels_have_tags = 0

for index, row in mt.iterrows():
    kernel_id = row["id"]
    tag_string = row["km_tags"]
    if not isinstance(tag_string, str):
        how_many_kernels_dont_have_tags += 1
        continue
    how_many_kernels_have_tags += 1
    tags_of_this_kernel = tag_string.split(";")
    for tag in tags_of_this_kernel:
        #####
        if tag in tags_kernel:
            tags_kernel[tag].append(kernel_id)
        else:
            tags_kernel[tag] = [kernel_id]
    kernel_tags[kernel_id] = tags_of_this_kernel
            
print(
    "how many kernels have tags: ", how_many_kernels_have_tags, "\n",  
    "how_many_kernels_dont_have_tags: ", how_many_kernels_dont_have_tags, "\n"
    #"how many tags are in total: ", len(list(tags_num.keys())), "\n"
)

how many kernels have tags:  6075 
 how_many_kernels_dont_have_tags:  22130 



In [ ]:
# 各个tag出现的kernel数
# 有各种tag的kernel的平均赞数是多少
# 什么tag往往会一起出现： frequent item set。

## Yangguang's tool for AST. 

In [11]:
import sys
import re
import ast
import json
from _ast import *

def classname(cls):
    return cls.__class__.__name__

def jsonify_ast(node, Type, level=0):

    #AttributeError: 'str' object has no attribute '_fields'
    if isinstance(node, str):
        return node

    fields = {}
    for k in node._fields:
        fields[k] = "..."
        v = getattr(node, k)
        if isinstance(v, ast.AST):
            if v._fields:
                fields[k] = jsonify_ast(v, Type)
            else:
                fields[k] = classname(v)

        elif isinstance(v, list):
            fields[k] = []
            for e in v:
                fields[k].append(jsonify_ast(e, Type))

        elif isinstance(v, str):
            fields[k] = v

        elif isinstance(v, int) or isinstance(v, float):
            fields[k] = v

        elif v is None:
            fields[k] = None

        else:
            fields[k] = 'unrecognized'
            print('Warming: unrecognized type detected.')

    if Type == 1:
        if isinstance(node, ast.expr) or isinstance(node, ast.stmt):
            line_number = node.lineno
            ret = { 'line': line_number, classname(node): fields }
        else:
            ret = { classname(node): fields }
    else:
        ret = { classname(node): fields }

    return ret

def gen_ast(path, Type):
    try:
        fileIn = open(path, "r")
        source = fileIn.read()
        AST = ast.parse(source_convert(source), '<unknow>', 'exec')

        a= jsonify_ast(AST, Type)
        fileIn.close()
        return json.dumps(a)
    except UnicodeDecodeError:
        print ("except UnicodeDecodeError")
        return ""


def gen_ast_from_source(source, Type):
    try:
        AST = ast.parse(source_convert(source), '<unknow>', 'exec')
    except SyntaxError:
        print ("except SyntaxError")
        return ""

    a= jsonify_ast(AST, Type)
    try:
        return json.dumps(a)
    except UnicodeDecodeError:
        print ("except UnicodeDecodeError")
        return ""


def source_convert(source):
    strinfo = re.compile(r'\\x')
    source = strinfo.sub('\\scale-unicode',source)
    strinfo1 = re.compile(r'-0\.')
    source = strinfo1.sub('1',source)
    return source

#cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script_clean_result"
#file_path = 'j:/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py'
file_path = "test_py.py"
with open(file_path, "r") as f:
    source = "".join(f.readlines())
AST = ast.parse(source, '<unknow>', 'exec')
a= jsonify_ast(AST, 1)
with open("nima.json", "w") as f:
    json.dump(a, f, indent= 4)
    a_s = json.dumps(a)

In [1]:
# import pygount, ast
# from _ast import *
# #---------------------------------------
# file_path__ = "test_py.py"
# #-----------

# print(parse_code(file_path__))
# #----------------------------------------
# print(import_list)
# print(import_from_list)
# print(line_no.keys())

[45, 6, 6, 9, 'csv;os;shutil;time;tqdm;wget', 'os.path.join']


## count sloc and corrensponding metrics for py code using pygount
This code snippet has been utilized. 

In [20]:
import pygount
analysis = pygount.source_analysis('j:/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py', 
                                   "j:/EECS_6414/Data/19-02-24/code_split/voted/python_script/0")
print(analysis)
sloc = analysis[3]
comment_rough = analysis[4]
empty = analysis[5]

SourceAnalysis(path='j:/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py', language='Python', group='j:/EECS_6414/Data/19-02-24/code_split/voted/python_script/0', code=422, documentation=216, empty=286, string=7, state='analyzed', state_info=None)


## Some metrics of code: generated from .ipynb scripts. 
### and we will also parse .py files. But those return values of .py files will all be 0. 

In [1]:
import json

path__ = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script\0\ahmadjaved097_classifying-heart-disease-patients.py"
#path__ = r"J:\CrawlerForModelStore\Overall\Overlap_Situation.ipynb"


print(parse_notebook(path__))

[0, 0, 0, 0, 0, 0]


In [2]:
import json, os, shutil, re, time
from lxml import etree
from tqdm import tqdm
import pygount, ast
from _ast import *


def parse_notebook(path):
    num_of_cell = 0
    num_of_output = 0
    ####据统计，就只有这么四种输出了。
    output_type_count = {"display_data": 0, "stream": 0, "execute_result": 0, "error": 0}  #
    try:
        #####################
        ## why try here?
        ## because if it is not a notebook, it will not open by json.load, and it will return error.
        ## retunr error, we will just simply return multiple 0s.
        #####################
        with open(path, "r", encoding="utf-8") as f:
            j = json.load(f)
        for cell in j["cells"]:
            num_of_cell += 1
            if "outputs" not in cell:
                continue
            for output in cell["outputs"]:
                num_of_output += 1
                output_type = output["output_type"]
                if output_type in output_type_count:
                    output_type_count[output_type] += 1
    except:
        pass

    new_list = [num_of_cell, num_of_output]
    new_list.extend(list(output_type_count.values()))
    return new_list


Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"

cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code_clean_result\measures_from_origin"
competition_file = os.path.join(cleaning_result_path, "code_stats_origin_{}.csv")
if not os.path.exists(cleaning_result_path):
    os.makedirs(cleaning_result_path)

log_file = os.path.join(cleaning_result_path, "logs.txt")
process_file = os.path.join(cleaning_result_path,
                            "process.txt")  # time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "kernel,author,num_of_cell,num_of_output,display_data,stream,execute_result,error\n"
START_NUMBERS = [i for i in range(0, 300 + 1)]

counter = 0

flag = True
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
            os.path.join(Author_path, "{}".format(folder))
    ):
        continue
    c = open(competition_file.format(folder), "w", encoding="utf-8")
    c.write(header_info)  # ",kernels" was the last feature.
    files_path = os.path.join(Author_path, "{}".format(folder))
    for page_name in (os.listdir(files_path)):
        try:
            suffix = page_name.split(".")[-1]
            if "py" not in suffix:
                continue
            #########################
            #             if (".ipynb" in page_name or ".xpynb" in page_name):
            #                 # something related to python notebook

            #                 pass
            #             elif ".py" in page_name:
            #                 # something related to py script

            #                 pass
            parse_result = parse_notebook(
                os.path.join(files_path, page_name)
            )
            c.write("{},{},".format(page_name.split("_")[-1].split(".")[0], page_name.split("_")[-2]) +
                    ",".join(["{}".format(item) for item in parse_result]) +
                    "\n")
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                   folder
                                   )
                  )
    c.close()
    print("The {} is хорошо.".format(folder))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

The 0 is хорошо.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


## go through the python code files

In [ ]:
import json, os, shutil, re, time, json
from lxml import etree
from tqdm import tqdm

Author_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\origin_code"
#cleaning_result_path = r"J:\EECS_6414\Data\19-02-24\code_split\voted\python_script_clean_result"
#competition_file = os.path.join(cleaning_result_path, "code_stats_{}.csv")
log_file = os.path.join(Author_path, "logs.txt")
process_file = os.path.join(Author_path, "process.txt") #time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
header_info = "kernel,author,attribute0\n"
START_NUMBERS = [i for i in range(0,300 + 1)]

counter = 0

flag = True
output_type = {}
for folder in tqdm(START_NUMBERS):
    if not os.path.isdir(
        os.path.join(Author_path, "{}".format(folder))
    ):
        continue
#     c = open(competition_file.format(folder), "w", encoding="utf-8")
#     c.write(header_info)  # ",kernels" was the last feature.
    files_dir = os.path.join(Author_path, "{}".format(folder))
    for page_name in tqdm(os.listdir(files_dir)):
        try:
#             c.write("{},{},{}\n".format(
#                 page_name.split("_")[-1].split(".")[0], page_name.split("_")[-2], "DMC5No1"
#             )
#             )
#------------------------------------------
            with open(os.path.join(files_dir, page_name), "r", encoding= "utf-8") as ff:
                j = json.load(ff)
            for cell in j["cells"]:
                if "outputs" not in cell:
                    continue
                for output in cell["outputs"]:
                    output_type[output["output_type"]] = -1
        except:
            with open(log_file, "a") as log:
                log.write("{},{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                              folder,
                                              page_name)
                          )
            continue
    with open(process_file, "a") as pro:
        pro.write("{},{}\n".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
                                folder
                                )
                  )
#     c.close()
    print("The {} is хорошо.".format(folder))
print(output_type.keys())

## use sloccount(installed in ubuntu) to count the sloc. 

In [12]:
# import os
# print(os.system("sloccount /mnt/j/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py"))
import subprocess

output = subprocess.run("sloccount /mnt/j/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py", shell=True, stdout=subprocess.PIPE, 
                        universal_newlines=True)
print(output.stdout)

Have a non-directory at the top, so creating directory top_dir
Adding /mnt/j/EECS_6414/Data/19-02-24/code_split/voted/python_script/0/abhishek_approaching-almost-any-nlp-problem-on-kaggle.py to top_dir
Categorizing files.
Finding a working MD5 command....
Found a working MD5 command.
Computing results.


SLOC	Directory	SLOC-by-Language (Sorted)
423     top_dir         python=423


Totals grouped by language (dominant language first):
python:         423 (100.00%)




Total Physical Source Lines of Code (SLOC)                = 423
Development Effort Estimate, Person-Years (Person-Months) = 0.08 (0.97)
 (Basic COCOMO model, Person-Months = 2.4 * (KSLOC**1.05))
Schedule Estimate, Years (Months)                         = 0.21 (2.47)
 (Basic COCOMO model, Months = 2.5 * (person-months**0.38))
Estimated Average Number of Developers (Effort/Schedule)  = 0.39
Total Estimated Cost to Develop                           = $ 10,947
 (average salary = $56,286/year, overhead = 2.40).
SLOCCount, Copyr

## ip crawler and verifier. 

For now I only find one website that IPs can be crawled from it: data5u.com. 

此代码已被融合进scrapy爬虫中。这个牛逼哄哄的爬虫在dropbox里面。

In [1]:
# *-* coding:utf-8 *-*
# https://blog.csdn.net/Gooooa/article/details/74452203
import requests
from lxml import etree

CHROME_PATH = "C:\\Users\\xmk233\\Downloads\\chromedriver.exe"

class Proxies(object):
    """docstring for Proxies"""
    def __init__(self, verify = False):
        self.ips = []
        self.protocals = []
        self.ports = []
        self.proxies = []
        self.all_proxies = []
        self.headers = {
        'Accept': '*/*',
        'User-Agent': '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"',
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'zh-CN,zh;q=0.8'
        }
        self.get_proxies()
        if verify:
            self.verify_proxies()

    def get_proxies(self):
        xpaths = {
            "ip":
                "//ul[@class='l2']/span[1]/li/text()",
            "port":
                "//ul[@class='l2']/span[2]/li/text()",
            "isHttps":
                "//ul[@class='l2']/span[4]/li/text()"
        }
#         option = Options()
#         option.add_argument('--headless')
#         webdriver = Chrome(executable_path= CHROME_PATH, chrome_options=option)
        
        for t in ["gngn", "gnpt", "gwgn", "gwpt"]:
            print(t)
            url = "http://www.data5u.com/free/{}/index.shtml".format(t)
#             html = webdriver.request('GET', url).content
            html = requests.get(url, headers=self.headers).content
            page_source = etree.HTML(html)
            self.ips.extend(page_source.xpath(xpaths["ip"]))
            self.ports.extend(page_source.xpath(xpaths["port"]))
            self.protocals.extend(page_source.xpath(xpaths["isHttps"]))

    def verify_proxies(self):
        print("start verifying...")
        for ip, port, isHttp in zip(self.ips, self.ports, self.protocals):
            protocol = isHttp
            proxy = "{}://{}:{}".format(isHttp, ip, port)
            proxies = {protocol: proxy}
            try:
                if requests.get('http://www.baidu.com', proxies=proxies, timeout=2).status_code == 200:
                    print ('success %s' % proxy)
                    self.proxies.append({"ipaddr": proxy})
            except:
                print ('fail %s' % proxy)
                #elf.failed_proxies.append({"ipaddr": proxy})
            self.all_proxies.append({"ipaddr": proxy})
                
if __name__ == '__main__':
    a = Proxies(True)
    IPPOOL = a.proxies if a.proxies != [] else a.all_proxies
    print(IPPOOL)

gngn
gnpt
gwgn
gwpt
start verifying...
fail http://183.148.154.223:8152
fail http://171.80.175.171:8772
fail http://111.79.198.231:8476
fail http://183.148.147.160:8920
fail http://59.62.165.86:8101
fail http://125.126.204.254:8249
fail http://59.62.164.88:8793
fail http://116.209.56.88:8254
fail http://27.29.72.112:8297
fail http://121.63.199.161:8912
fail http://116.209.57.144:8234
fail http://115.46.89.167:8361
success https://121.61.1.197:8934
success https://116.209.56.116:8951
fail http://59.62.165.203:8677
fail http://221.11.105.71:8146
success https://14.115.105.46:8699
success https://113.78.255.149:8928
success https://119.123.244.173:8530
success https://118.114.210.234:8083
fail http://101.236.41.207:8081
success https://59.62.167.152:8121
fail http://117.66.166.97:8882
fail http://114.249.116.88:8455
fail http://123.147.165.143:8590
fail http://123.147.165.144:8138
fail http://14.20.235.195:8280
success https://125.123.143.175:8661
fail http://119.139.196.38:8653
fail http

In [21]:
from seleniumrequests import Chrome
from selenium.webdriver.chrome.options import Options
from lxml import etree
CHROME_PATH = "C:\\Users\\xmk233\\Downloads\\chromedriver.exe"
headers = {
        'Accept': '*/*',
        'User-Agent': '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"',
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'zh-CN,zh;q=0.8'
        }
url = "http://www.data5u.com/free/gwgn/index.shtml"
option = Options()
option.add_argument('--headless')
webdriver = Chrome(executable_path= CHROME_PATH, chrome_options=option)
html = webdriver.request('GET', url).content
page_source = etree.HTML(html)
xpaths = {
    "ip":
        "//ul[@class='l2']/span[1]/li/text()",
    "port":
        "//ul[@class='l2']/span[2]/li/text()",
    "isHttps":
        "//ul[@class='l2']/span[4]/li/text()"
}
ips = page_source.xpath(xpaths["ip"])
ports = page_source.xpath(xpaths["port"])
isHttps = page_source.xpath(xpaths["isHttps"])
for ip, port, isHttp in zip(ips, ports, isHttps):
    self.proxies.append(
        "{}://{}:{}".format(isHttp, ip, port)
    )
print(ips, ports, isHttps)

['176.110.154.119', '91.90.37.94', '89.218.22.178', '223.16.229.241', '1.20.101.46', '185.3.68.239', '200.107.35.154', '78.186.236.138', '45.231.30.201', '83.212.107.68', '86.57.219.182', '128.199.217.17', '103.210.121.1', '186.178.10.158', '13.234.49.74'] ['8610', '8328', '8207', '8199', '8385', '8404', '8993', '8344', '8379', '9069', '8936', '9077', '8521', '8195', '8866'] ['http', 'http', 'http', 'http', 'http', 'http', 'http', 'http', 'http', 'http', 'http', 'https', 'http', 'http', 'http']
